In [1]:
import os

base_folder = 'app/DashApp/HeartSound'
conditions_list = [
    "Normal Heart",
    "Split First Heart",
    "Split Second Heart",
    "Third Heart",
    "Fourth Heart",
    "Functional Murmur",
    "Diastolic Murmur",
    "Opening Snap",
    "Holosystolic Murmur",
    "Early Systolic Murmur",
    "Mid Systolic Murmur",
    "Continuous Murmur",
    "Austin Flint Murmur",
    "Pericardial Rub",
    "Graham Steell Murmur",
    "Aortic Valve Regurgitation",
    "Aortic Valve Stenosis",
    "Aortic Valve Stenosis Regurgitation",
    "Congenital Aortic Stenosis",
    "Mitral Valve Regurgitation",
    "Mitral Valve Stenosis",
    "Mitral Valve Prolapse",
    "Mitral Stenosis Regurgitation",
    "Mitral Stenosis Tricuspid Regurgitation",
    "Pulmonary Valve Stenosis",
    "Pulmonary Valve Regurgitation",
    "Tricuspid Valve Regurgitation",
    "Coarctation of the Aorta",
    "Hypertrophic Cardiomyopathy",
    "Patent Ductus Arteriosus",
    "Atrial Septal Defect",
    "Ventricular Septal Defect",
    "Acute Myocardial Infarction",
    "Congestive Heart Failure",
    "Systemic Hypertension",
    "Acute Pericarditis",
    "Dilated Cardiomyopathy",
    "Pulmonary Hypertension",
    "Tetralogy of Fallot",
    "Ventricular Aneurysm",
    "Ebstein Anomaly"
]

# Create main folder
os.makedirs(base_folder, exist_ok=True)

# Create sub-folders and files
for condition in conditions_list:
    condition_folder = os.path.join(base_folder, condition)
    os.makedirs(condition_folder, exist_ok=True)
    
    for filename in ['mitral.py', 'aortic.py', 'pulmonary.py', 'tricuspid.py', 'erb.py']:
        file_path = os.path.join(condition_folder, filename)
        with open(file_path, 'w') as file:
            file.write("# Placeholder content for {}".format(filename))

print("Folder structure and files created successfully.")

Folder structure and files created successfully.


In [3]:
import os

base_folder = 'app/DashApp/HeartSound'
conditions_list = [
    "Normal Heart",
    "Split First Heart",
    "Split Second Heart",
    "Third Heart",
    "Fourth Heart",
    "Functional Murmur",
    "Diastolic Murmur",
    "Opening Snap",
    "Holosystolic Murmur",
    "Early Systolic Murmur",
    "Mid Systolic Murmur",
    "Continuous Murmur",
    "Austin Flint Murmur",
    "Pericardial Rub",
    "Graham Steell Murmur",
    "Aortic Valve Regurgitation",
    "Aortic Valve Stenosis",
    "Aortic Valve Stenosis Regurgitation",
    "Congenital Aortic Stenosis",
    "Mitral Valve Regurgitation",
    "Mitral Valve Stenosis",
    "Mitral Valve Prolapse",
    "Mitral Stenosis Regurgitation",
    "Mitral Stenosis Tricuspid Regurgitation",
    "Pulmonary Valve Stenosis",
    "Pulmonary Valve Regurgitation",
    "Tricuspid Valve Regurgitation",
    "Coarctation of the Aorta",
    "Hypertrophic Cardiomyopathy",
    "Patent Ductus Arteriosus",
    "Atrial Septal Defect",
    "Ventricular Septal Defect",
    "Acute Myocardial Infarction",
    "Congestive Heart Failure",
    "Systemic Hypertension",
    "Acute Pericarditis",
    "Dilated Cardiomyopathy",
    "Pulmonary Hypertension",
    "Tetralogy of Fallot",
    "Ventricular Aneurysm",
    "Ebstein Anomaly"
]

program_content = """
import dash
from urllib import request
from dash import dcc, html
from dash.dependencies import Input, Output, State
from pydub import AudioSegment
from django_plotly_dash import DjangoDash
import numpy as np
import pandas as pd
import time
import sqlite3

# Create Dash app
app = DjangoDash('hbrDash')
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()
df = pd.read_sql_query("SELECT * FROM app_heartaudio", con)

audio_path = df.loc[(df['sound_name'] == 'normal_heart') & (df['sound_type'] == 'M'), 'audio_file_path'].values[0]
audio = AudioSegment.from_file(audio_path)
audio_data = np.array(audio.get_array_of_samples())
audio_duration = len(audio_data) / audio.frame_rate
subsampling_factor = 1
audio_data = audio_data[::subsampling_factor]

target_duration = 0.877
num_repeats = 1
audio_data = np.tile(audio_data, num_repeats)
sample_rate = 44100
duration = len(audio_data) / sample_rate

# Layout of the app
app.layout = html.Div([
    dcc.Graph(id='animated-audio-chart', style={'height': '95vh'}, config={'responsive': True}),
    dcc.Store(id='audio-data-store', data={'audio_data': audio_data.tolist(), 'audio_duration': duration}),
    dcc.Store(id='interval-store', data=time.time()),  # Store the start time and set default heart rate to 60
    dcc.Interval(id='interval-component', interval=25, n_intervals=0) # Interval in milliseconds
])

# Clientside callback to update the graph
app.clientside_callback(
    \"""
    function(n, audioData, startTime) {
        // Get the audio data and duration from the stored data
        var audioArray = audioData['audio_data'];
        var audioDuration = audioData['audio_duration'];
        var heartRate = 60

        // Calculate the time passed since the start
        var currentTime = new Date().getTime() / 1000;  // Convert milliseconds to seconds
        var elapsedTime = currentTime - startTime;

        // Calculate the position of the vertical line
        var timePerBeat = 60 / heartRate;
        var linePosition = Math.floor((elapsedTime % timePerBeat) * audioArray.length / timePerBeat);

        // Create the figure
        var figure = {
            data: [
                {y: audioArray, type: 'line', name: 'HBR Signal', line: {color: 'green'}},
                {x: [linePosition, linePosition], y: [Math.min.apply(null, audioArray), Math.max.apply(null, audioArray)], mode: 'lines', line: {color: 'black', width: 10}}
            ],
            layout: {
                showlegend: false,
                paper_bgcolor: 'black',  // Set background color to black
                plot_bgcolor: 'black'    // Set plot background color to black
            }
        };
        
        // Return the updated figure, startTime, and heartRate
        return [figure, startTime];
    }
    \""",
    Output('animated-audio-chart', 'figure'),
    Output('interval-store', 'data'),
    Input('interval-component', 'n_intervals'),
    State('audio-data-store', 'data'),
    State('interval-store', 'data'),
    prevent_initial_call=True
)
"""

# Create main folder
os.makedirs(base_folder, exist_ok=True)

# Create sub-folders and files
for condition in conditions_list:
    condition_folder = os.path.join(base_folder, condition)
    os.makedirs(condition_folder, exist_ok=True)
    
    for filename in ['mitral.py', 'aortic.py', 'pulmonary.py', 'tricuspid.py', 'erb.py']:
        file_path = os.path.join(condition_folder, filename)
        with open(file_path, 'w') as file:
            file.write(program_content)

print("Folder structure and files created successfully.")

Folder structure and files created successfully.
